In [6]:
### インポート ###

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime
import time
from selenium.webdriver.common.keys import Keys
import re
import csv
import pandas as pd
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List

In [20]:
### 定数 ###

# 対象競馬場とレース
PLACE_CODES = [
    "35", "45", "55", "65", "46", "54", "44",
    "47", "50", "30", "36", "43", "48", "42"
]
DIC_STYLE = {
    1: "逃げ",
    2: "先行",
    3: "差し",
    4: "追込"
}
# レース番号
RACE_NUMBERS = [f"{i:02d}" for i in range(1, 13)]
# スクレイピング
PATH_CHROME_DRIVER = 'C:\\Users\\ryo\\AppData\\Local\\SeleniumBasic\\chromedriver.exe'
LOGIN_URL = "https://regist.netkeiba.com/account/?pid=login"
LOGIN_ID = "keibaaitool@gmail.com"
LOGIN_PASSWORD = "keibaai"
RACE_LIST_URL = "https://nar.netkeiba.com/top/race_list.html?kaisai_date="
HORSE_NUMBER_XPATH_1 = "//*[@id='tr_"
HORSE_NUMBER_XPATH_2 = "']/td[2]"
HORSE_LINK_XPATH_1_1 = "/html/body/div[1]/div[3]/div[3]/table/tbody/tr["
HORSE_LINK_XPATH_1_2 = "]/td[4]/div/div/span[2]/a"
HORSE_LINK_XPATH_2_1 = "/html/body/div[1]/div[3]/div[3]/table/tbody/tr["
HORSE_LINK_XPATH_2_2 = "]/td[4]/div/div/span/a"
JOCKEY_LINK_XPATH_1 = "//*[@id='tr_"
JOCKEY_LINK_XPATH_2 = "']/td[7]/span/a"
POPULARITY_XPATH_1 = "//*[@id='tr_"
POPULARITY_XPATH_2 = "']/td[11]"
ODDS_XPATH_1 = "//*[@id='tr_"
ODDS_XPATH_2 = "']/td[10]"
IMPOSE_XPATH_1 = "//*[@id='tr_"
IMPOSE_XPATH_2 = "']/td[6]"
FRAME_XPATH_1 = "//*[@id='tr_"
FRAME_XPATH_2 = "']/td[1]"
WEIGHT_XPATH_1 = "//*[@id='tr_"
WEIGHT_XPATH_2 = "']/td[9]"
SEX_AND_AGE_XPATH_1 = "//*[@id='tr_"
SEX_AND_AGE_XPATH_2 = "']/td[5]/span"
LAST_3_FURLONGS_PRED_XPATH_1 = "//*[@id='ai_tenkai']/div/section/div[3]/table[1]/tbody/tr["
LAST_3_FURLONGS_PRED_XPATH_2 = "]/td[3]"
RACECOURSE_SELECTOR = "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(2)"
RACE_NUMBER_SELECTOR = "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item01 > div"
NUM_HORSES_SELECTOR = "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(5)"
RACE_INFO_XPATH = "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[2]/div/div[1]/div[3]/div[2]/div[2]"
RELIABILITY_XPATH = "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[12]/table/tbody/tr[2]/td/p/span"
OPINION_XPATH = "//*[@id='ai_tenkai']/div/section/div[4]/section/dl/dd[1]/p"
CORNER_1_XPATH = "//*[@id='Corner01']/span[1]"
CORNER_2_XPATH = "//*[@id='Corner02']/span[1]"
CORNER_3_XPATH = "//*[@id='Corner03']/span[1]"

In [22]:
### クラス ###

# 馬データ
@dataclass
class Horse:
    """出走馬のデータを保持するデータクラス"""
    horse_id: str
    horse_name: str
    jockey_id: str
    jockey_name: str
    popularity: str
    odds: str
    sex_and_age: str
    weight: str
    horse_number: str
    frame_number: str
    position_1_top_pred: str
    position_1_left_pred: str
    position_2_top_pred: str
    position_2_left_pred: str
    position_3_top_pred: str
    position_3_left_pred: str
    position_4_top_pred: str
    position_4_left_pred: str 
    position_1_top_pred_jockey_tendency: str
    position_1_left_pred_jockey_tendency: str
    position_2_top_pred_jockey_tendency: str
    position_2_left_pred_jockey_tendency: str
    position_3_top_pred_jockey_tendency: str
    position_3_left_pred_jockey_tendency: str
    position_4_top_pred_jockey_tendency: str
    position_4_left_pred_jockey_tendency: str
    style_pred: str
    impost: str
    last_3_furlongs_pred: str

# レースデータ
@dataclass
class Race:
    """レースの基本情報と出走馬リストを保持するデータクラス"""
    race_id: str
#    race_name: str
    race_date: str
    race_time: str
    num_horses: str
    race_number: int
#    weather_name: str
#    track_condition_name: str
    racecourse: str
    ground: str
    distance: str
    direction:  str
    reliability: str
    opinion: str
    horses: List[Horse] = field(default_factory=list)

In [45]:
### 関数 ###

def is_xpath_present(driver, xpath):
    return len(driver.find_elements(By.XPATH, xpath)) > 0

def extract_style_values(style_str):
    """
    style文字列からtopとleftの数値を抽出する関数
    """
    # 正規表現の解説:
    # -?    : マイナス記号があってもなくても良い
    # [\d\.]+ : 数字またはドットが1回以上続く
    # ()    : このカッコ内の部分を抽出する
    top_match = re.search(r"top:\s*(-?[\d\.]+)%", style_str)
    left_match = re.search(r"left:\s*(-?[\d\.]+)%", style_str)
    # 抽出した文字列をfloatに変換
    top_val = float(top_match.group(1)) if top_match else None
    left_val = float(left_match.group(1)) if left_match else None
    # 戻り値
    return top_val, left_val

def extract_style_values(style_str):
    """
    style文字列からtopとleftの数値を抽出する関数（マイナス対応版）
    """
    # 正規表現の解説:
    # -?    : マイナス記号があってもなくても良い
    # [\d\.]+ : 数字またはドットが1回以上続く
    # ()    : このカッコ内の部分を抽出する
    top_match = re.search(r"top:\s*(-?[\d\.]+)%", style_str)
    left_match = re.search(r"left:\s*(-?[\d\.]+)%", style_str)
    
    # 抽出した文字列をfloatに変換
    top_val = float(top_match.group(1)) if top_match else None
    left_val = float(left_match.group(1)) if left_match else None
    
    return top_val, left_val

def get_position_pred(driver):
    # HorseIconクラスを持つすべての要素を取得（リスト形式で返ってくる）
    position_pred_elements = driver.find_elements(By.CLASS_NAME, "HorseIcon")
    # 結果を格納する辞書
    dic_position_pred = {}
    for element in position_pred_elements:
        horse_number = element.get_attribute("id")
        position_pred = element.get_attribute("style")
        top_val, left_val = extract_style_values(position_pred)
        dic_val = {}
        dic_val["top"] = top_val
        dic_val["left"] = left_val
        # idが空でない場合のみ追加
        if horse_number:
            dic_position_pred[horse_number] = dic_val
    # 戻り値
    return dic_position_pred

def login_netkeiba(driver):
    # ログイン
    driver.get(LOGIN_URL)
    driver.find_element(By.NAME, "login_id").send_keys(LOGIN_ID)
    pw_field = driver.find_element(By.NAME, "pswd")
    pw_field.send_keys(LOGIN_PASSWORD)
    pw_field.send_keys(Keys.ENTER)
    time.sleep(3)

def scrape_horse_data(driver, num_horses):
    
    all_horses_data = []
    # 馬番・馬名取得
    # 馬番・馬名・馬名リンク取得
    for i in range(1, num_horses + 1):
        try:
            # 馬番
            horse_number_xpath = HORSE_NUMBER_XPATH_1 + str(i) + HORSE_NUMBER_XPATH_2
            if is_xpath_present(driver, horse_number_xpath):
                horse_number = driver.find_element(By.XPATH, horse_number_xpath).text.strip()
            else:
                break  
            # 馬名と馬ID
            horse_link_xpath_1 = HORSE_LINK_XPATH_1_1 + str(i) + HORSE_LINK_XPATH_1_2
            horse_link_xpath_2 = HORSE_LINK_XPATH_2_1 + str(i) + HORSE_LINK_XPATH_2_2
            if is_xpath_present(driver, horse_link_xpath_1):
                horse_link_xpath = horse_link_xpath_1
            elif is_xpath_present(driver, horse_link_xpath_2):
                horse_link_xpath = horse_link_xpath_2
            horse_link_elem = driver.find_element(By.XPATH, horse_link_xpath)
            horse_name = horse_link_elem.text.strip()
            horse_href = horse_link_elem.get_attribute("href")
            horse_id = horse_href.rstrip("/").split("/")[-1]
            # 騎手名と騎手ID
            jockey_link_xpath = JOCKEY_LINK_XPATH_1 + str(i) + JOCKEY_LINK_XPATH_2
            jockey_link_elem = driver.find_element(By.XPATH, jockey_link_xpath)
            jockey_name = jockey_link_elem.text.strip()
            jockey_href = jockey_link_elem.get_attribute("href")
            jockey_id = jockey_href.rstrip("/").split("/")[-1]
            # 人気
            popularity_xpath = POPULARITY_XPATH_1 + str(i) + POPULARITY_XPATH_2
            popularity = driver.find_element(By.XPATH, popularity_xpath).text
            # オッズ
            odds_xpath = ODDS_XPATH_1 + str(i) + ODDS_XPATH_2
            odds = driver.find_element(By.XPATH, odds_xpath).text
            # 性齢
            sex_and_age_xpath = SEX_AND_AGE_XPATH_1 + str(i) + SEX_AND_AGE_XPATH_2
            sex_and_age = driver.find_element(By.XPATH, sex_and_age_xpath).text
            # 馬体重
            weight_xpath = WEIGHT_XPATH_1 + str(i) + WEIGHT_XPATH_2
            weight = driver.find_element(By.XPATH, weight_xpath).text
            # 枠番
            frame_number_xpath = FRAME_XPATH_1 + str(i) + FRAME_XPATH_2
            frame_number = driver.find_element(By.XPATH, frame_number_xpath).text
            # 斤量
            impost_xpath = IMPOSE_XPATH_1 + str(i) + IMPOSE_XPATH_2
            impost = driver.find_element(By.XPATH, impost_xpath).text
            # 後半3F
            last_3_furlongs_pred_xpath = LAST_3_FURLONGS_PRED_XPATH_1 + str(i) + LAST_3_FURLONGS_PRED_XPATH_2
            last_3_furlongs_pred = driver.find_element(By.XPATH, last_3_furlongs_pred_xpath).text
            
            horse_data = Horse(
                horse_id = horse_id,
                horse_name = horse_name,
                jockey_id = jockey_id,
                jockey_name = jockey_name,
                popularity = popularity,
                odds = odds,
                sex_and_age = sex_and_age,
                weight = weight,
                horse_number = horse_number,
                frame_number = frame_number,
                position_1_top_pred = None,
                position_1_left_pred = None,
                position_2_top_pred = None,
                position_2_left_pred = None,
                position_3_top_pred = None,
                position_3_left_pred = None,
                position_4_top_pred = None,
                position_4_left_pred = None,
                position_1_top_pred_jockey_tendency = None,
                position_1_left_pred_jockey_tendency = None,
                position_2_top_pred_jockey_tendency = None,
                position_2_left_pred_jockey_tendency = None,
                position_3_top_pred_jockey_tendency = None,
                position_3_left_pred_jockey_tendency = None,
                position_4_top_pred_jockey_tendency = None,
                position_4_left_pred_jockey_tendency = None,
                style_pred = None,
                impost = impost,
                last_3_furlongs_pred = last_3_furlongs_pred
            )
            
            all_horses_data.append(horse_data)
        except Exception as e:
            print(f"  馬番{i}: データ取得失敗 ({e})")
    # 戻り値
    return all_horses_data

def scrape_race_data(driver, race_id, date):

    try:
        # 競馬場
        racecourse = driver.find_element(By.CSS_SELECTOR, RACECOURSE_SELECTOR).text
        # R
        race_number = driver.find_element(By.CSS_SELECTOR, RACE_NUMBER_SELECTOR).text
        # 頭数
        num_horses = driver.find_element(By.CSS_SELECTOR, NUM_HORSES_SELECTOR).text
        num_horses = int(num_horses.replace("頭", "").strip())
        # レース情報（発走時刻、グラウンド、距離、向き）
        race_info = driver.find_element(By.XPATH, RACE_INFO_XPATH).text
        race_info = race_info.strip()
        # "/"で分割
        parts = [p.strip() for p in race_info.split("/")]            
        # 発走時刻を抽出
        race_time = parts[0].replace("発走", "").strip()
        ground_info = parts[1]
        # グラウンドを抽出
        ground_match = re.search(r"(芝|ダ)", ground_info)
        ground = ground_match.group(1) if ground_match else "不明"
        # 距離を抽出
        distance_match = re.search(r"(\d+)m", ground_info)
        distance = int(distance_match.group(1)) if distance_match else -1
        # 向きを抽出
        direction_match = re.search(r"\((左|右)\)", ground_info)
        direction = direction_match.group(1) if direction_match else "不明"
        # 1人気信頼度
        reliability = driver.find_element(By.XPATH, RELIABILITY_XPATH).text
        # 見解
        opinion = driver.find_element(By.XPATH, OPINION_XPATH).text
        # 馬のデータを抽出
        horse_list = scrape_horse_data(driver, num_horses)
        # ポジション予想（1）
        corner = driver.find_element(By.XPATH, CORNER_1_XPATH).text
        dic_position_pred = get_position_pred(driver)
        match corner:
            case "スタート後":
                for horse in horse_list:
                    horse.position_1_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_1_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "2コーナー":
                for horse in horse_list:
                    horse.position_2_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_2_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "3コーナー":
                for horse in horse_list:
                    horse.position_3_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_3_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "4コーナー":
                for horse in horse_list:
                    horse.position_4_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_4_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
        # ポジション予想（2）
        driver.find_element(By.XPATH, CORNER_2_XPATH).click()
        corner = driver.find_element(By.XPATH, CORNER_2_XPATH).text
        dic_position_pred = get_position_pred(driver)
        # ポジション予想を馬と紐づける
        match corner:
            case "スタート後":
                for horse in horse_list:
                    horse.position_1_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_1_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "2コーナー":
                for horse in horse_list:
                    horse.position_2_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_2_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "3コーナー":
                for horse in horse_list:
                    horse.position_3_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_3_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "4コーナー":
                for horse in horse_list:
                    horse.position_4_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_4_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
        # ポジション予想（3）
        driver.find_element(By.XPATH, CORNER_3_XPATH).click()
        corner = driver.find_element(By.XPATH, CORNER_3_XPATH).text
        dic_position_pred = get_position_pred(driver)
        # ポジション予想を馬と紐づける
        match corner:
            case "スタート後":
                for horse in horse_list:
                    horse.position_1_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_1_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "2コーナー":
                for horse in horse_list:
                    horse.position_2_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_2_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "3コーナー":
                for horse in horse_list:
                    horse.position_3_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_3_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "4コーナー":
                for horse in horse_list:
                    horse.position_4_top_pred = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_4_left_pred = dic_position_pred[f'Horse{horse.horse_number}']["left"]
        # 「騎手傾向を考慮」をチェック
        driver.find_element(By.XPATH, "//*[@id='dev_check_01_03']").click()
        # ポジション予想（1）
        driver.find_element(By.XPATH, CORNER_1_XPATH).click()
        corner = driver.find_element(By.XPATH, CORNER_1_XPATH).text
        dic_position_pred = get_position_pred(driver)
        match corner:
            case "スタート後":
                for horse in horse_list:
                    horse.position_1_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_1_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "2コーナー":
                for horse in horse_list:
                    horse.position_2_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_2_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "3コーナー":
                for horse in horse_list:
                    horse.position_3_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_3_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "4コーナー":
                for horse in horse_list:
                    horse.position_4_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_4_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
        # ポジション予想（2）
        driver.find_element(By.XPATH, CORNER_2_XPATH).click()
        corner = driver.find_element(By.XPATH, CORNER_2_XPATH).text
        dic_position_pred = get_position_pred(driver)
        # ポジション予想を馬と紐づける
        match corner:
            case "スタート後":
                for horse in horse_list:
                    horse.position_1_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_1_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "2コーナー":
                for horse in horse_list:
                    horse.position_2_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_2_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "3コーナー":
                for horse in horse_list:
                    horse.position_3_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_3_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "4コーナー":
                for horse in horse_list:
                    horse.position_4_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_4_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
        # ポジション予想（3）
        driver.find_element(By.XPATH, CORNER_3_XPATH).click()
        corner = driver.find_element(By.XPATH, CORNER_3_XPATH).text
        dic_position_pred = get_position_pred(driver)
        # ポジション予想を馬と紐づける
        match corner:
            case "スタート後":
                for horse in horse_list:
                    horse.position_1_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_1_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "2コーナー":
                for horse in horse_list:
                    horse.position_2_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_2_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "3コーナー":
                for horse in horse_list:
                    horse.position_3_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_3_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]
            case "4コーナー":
                for horse in horse_list:
                    horse.position_4_top_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["top"]
                    horse.position_4_left_pred_jockey_tendency = dic_position_pred[f'Horse{horse.horse_number}']["left"]                
        # 「旧版の展開予想に切り替える」をクリックして脚質予想を取得
        driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[14]/div/div/label/span").click()
        list_lead = []
        list_stalker = []
        list_chaser = []
        list_closer = []
        # 逃げ
        for i in range(1, num_horses + 1):
            if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[1]/td/div[" + str(i) + "]/span[1]"):
                list_lead.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[1]/td/div[" + str(i) + "]/span[1]").text)
            else:
                 break
        # 先行
        for i in range(1, num_horses + 1):
            if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[2]/td/div[" + str(i) + "]/span[1]"):
                list_stalker.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[2]/td/div[" + str(i) + "]/span[1]").text)
            else:
                 break
        # 差し
        for i in range(1, num_horses + 1):
            if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[3]/td/div[" + str(i) + "]/span[1]"):
                list_chaser.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[3]/td/div[" + str(i) + "]/span[1]").text)
            else:
                 break
        # 追込
        for i in range(1, num_horses + 1):
            if is_xpath_present(driver, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[4]/td/div[" + str(i) + "]/span[1]"):
                list_closer.append(driver.find_element(By.XPATH, "//*[@id='Netkeiba_Race_Nar_Shutuba']/div[1]/div[3]/div[16]/table/tbody/tr[4]/td/div[" + str(i) + "]/span[1]").text)
            else:
                 break
        # 脚質予想を馬と紐づける
        for horse in horse_list:
            if horse.horse_number in list_lead:
                horse.style_pred = DIC_STYLE[1]
            elif horse.horse_number in list_stalker:
                horse.style_pred = DIC_STYLE[2]
            elif horse.horse_number in list_chaser:
                horse.style_pred = DIC_STYLE[3]
            elif horse.horse_number in list_closer:
                horse.style_pred = DIC_STYLE[4]
        # レースデータに格納
        race_data = Race(
            race_id = race_id,
            # race_name
            race_date = date,
            race_time = race_time,
            num_horses = num_horses,
            race_number = race_number,
            # weather_name
            # track_condition_name,
            racecourse = racecourse,
            ground = ground,
            distance = distance,
            direction = direction,
            reliability = reliability,
            opinion = opinion,
            horses = horse_list
        )
        # 出力確認
        print(f"\n📄 Race ID: {race_data.race_id} | 競馬場: {race_data.racecourse} | R: {race_data.race_number} | 信頼度: {race_data.reliability}")
        for horse in race_data.horses:
            print(f"  馬番: {horse.horse_number} | 馬ID: {horse.horse_id} | 馬名: {horse.horse_name}")
            print(f"  脚質予想: {horse.style_pred}")
    
    except Exception as e:
        print(f"❌ Race ID: {race_id} データ取得に失敗: {e}")
        return None
    # 戻り値
    return race_data

def scrape_one_day_data(driver, date):
    """
    競馬場とRでループ
    """
    all_races_data = []
    # 競馬場のループ
    for place_code in PLACE_CODES:
        # Rのループ
        for race_num in RACE_NUMBERS:
            yyyy = date.strftime("%Y")
            mmdd = date.strftime("%m%d")            
            race_id = f"{yyyy}{place_code}{mmdd}{race_num}"
            race_url = f"https://nar.netkeiba.com/race/shutuba.html?race_id={race_id}"
            driver.get(race_url)
            time.sleep(1)
            race_data = scrape_race_data(driver, race_id, date)
            # 1Rのrace_dataがNoneだったらその競馬場では開催していないためRのループを抜けたい
            if race_data is None:
                break
            all_races_data.append(race_data)
    # 戻り値
    return all_races_data

def main_scraper(driver, date_list):
    """
    年月日でループ
    """
    all_days_data = []
    # 年月日のループ
    for date in date_list:
        # 年月日のURLでアクセス
        date_str = date.strftime("%Y%m%d")
        race_list_url = RACE_LIST_URL + date_str
        driver.get(race_list_url)
        one_day_data = scrape_one_day_data(driver, date)
        all_days_data.append(one_day_data)
    # 戻り値
    return all_days_data

def save_to_csv(final_result, filename="race_data.csv"):
    """
    スクレイピング結果（Raceオブジェクトのリスト）を平坦化してCSVに出力する
    """
    flattened_data = []

    # final_result は [ [Race1, Race2, ...], [Race1, Race2, ...] ] という2次元リスト
    for day_data in final_result:
        for race in day_data:
            # レース情報の共通項目を辞書化 (horsesリスト以外を取得)
            race_info = {
                "race_id": race.race_id,
                "race_date": race.race_date.strftime("%Y-%m-%d") if isinstance(race.race_date, datetime) else race.race_date,
                "race_time": race.race_time,
                "num_horses": race.num_horses,
                "race_number": race.race_number,
                "racecourse": race.racecourse,
                "ground": race.ground,
                "distance": race.distance,
                "direction": race.direction,
                "reliability": race.reliability,
                "opinion": race.opinion
            }

            # そのレースに属する馬のデータを結合
            for horse in race.horses:
                # 馬のデータを辞書化
                horse_info = vars(horse)
                
                # レース情報と馬情報を統合した新しい辞書を作成
                combined_dict = {**race_info, **horse_info}
                flattened_data.append(combined_dict)

    if not flattened_data:
        print("出力するデータがありません。")
        return

    # DataFrameに変換
    df = pd.DataFrame(flattened_data)

    # CSV出力 (日本語文字化け防止のため utf-8-sig を指定)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ CSV出力が完了しました: {filename}")

In [49]:
### メイン処理 ###

# 対象期間
start_date_str = "20250919"
end_date_str = "20250919"
# スクレイピング準備
service = Service(PATH_CHROME_DRIVER)
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=options)
# ログイン
login_netkeiba(driver)
# 開始日～終了日
date_list = pd.date_range(
    start=start_date_str,
    end=end_date_str,
    freq='D'
)
# データ取得
final_result = main_scraper(driver, date_list)
# CSVへ保存
save_to_csv(final_result, f"C:\\Users\\ryo\\Downloads\\keiba_data_{start_date_str}-{end_date_str}.csv")
# スクレイピング終了
driver.quit()

❌ Race ID: 202535091901 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202545091901 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202555091901 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202565091901 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202546091901 データ取得に失敗: invalid literal for int() with base 10: ''
❌ Race ID: 202554091901 データ取得に失敗: invalid literal for int() with base 10: ''

📄 Race ID: 202544091901 | 競馬場: 大井 | R: 1R | 信頼度: -%
  馬番: 1 | 馬ID: 2022101342 | 馬名: カリナ
  脚質予想: 先行
  馬番: 2 | 馬ID: 2022107045 | 馬名: クアッズ
  脚質予想: 差し
  馬番: 3 | 馬ID: 2022100428 | 馬名: ミュークエンペラー
  脚質予想: 追込
  馬番: 4 | 馬ID: 2022106993 | 馬名: アルミンテソーロ
  脚質予想: 差し
  馬番: 5 | 馬ID: 2022105806 | 馬名: アレグランサ
  脚質予想: 先行
  馬番: 6 | 馬ID: 2022100149 | 馬名: ケングロリア
  脚質予想: 追込
  馬番: 7 | 馬ID: 2022105932 | 馬名: テルケンレンスター
  脚質予想: 差し
  馬番: 8 | 馬ID: 2022103397 | 馬名: フロントドア
  脚質予想: 差し
  馬番: 9 | 馬ID: 2022100296 | 馬名: キセキヲノゾムナラ
  脚質予想: 先行
  馬番: 10 | 馬ID: 2022106786 | 

In [27]:
# 地方競馬 https://nar.netkeiba.com/top/race_list.html?kaisai_date=YYYYMMDD
# 波乱度 https://nar.sp.netkeiba.com/race/compatibility.html
# 調子偏差値 https://race.sp.netkeiba.com/barometer/profile.html?kaisai_date=YYYYMMDD